<a href="https://colab.research.google.com/github/LimDoHyeon/PKNU-MachineLearning/blob/main/%5BPKNU%5D_ML_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.2 합성곱의 기본 이해

- 합성곱 : 선형 필터링, 영상 처리 방법의 하나.
  - 커널(kernel) 혹은 필터라고 불리는 작은 행렬을 이용해 영상과 선형 결합으로 새로운 값을 얻는다.
  - SVM의 '커널'이라는 용어와 구별 -> 혼돈을 피하기 위해 이 행렬은 필터라고 부르기도 한다.

- 필터링 : 이미지를 변형하는 방식. 다음과 같이 이용
  - F : 필터, 작은 행렬.
  - R : F를 원본 이미지에 놓을 때, 중첩하는 영역. 부분 행렬, **관심 영역**.
  - F와 R의 대응되는 값들을 서로 곱한 뒤에 모두 더한 것 : 이러한 연산을 **합성곱 연산**, 혹은 **컨볼루션**이라고 함.

---

**여러 필터의 종류**
- 평균 필터 (= 상자 필터)
  - 가장 간단한 필터링 함수
  - 어떤 픽셀의 값을 변경할 때 주위 값과 평균을 취하는 방법

- 평활화 필터 (= 가우스 필터)
  - 필터 크기 범위 안의 모든 픽셀에 **중심 픽셀에는 높은 가중치(중요도)**, 중심에서 멀 수록 낮은 가중치를 두는 선형 결합
  - 가중치(weight)값의 분포는 정규분포 함수를 통해 그 값을 결정
  - 정규분포(가우스) 함수는 종 모양 함수 혹은 정규 분포 함수

# 9.5 패딩, 스트라이딩
- 패딩(padding)
  - 합성곱 연산을 하면 이미지의 크기가 작아짐
  - 이를 방지하기 위해 입력 이미지의 주변에 (필터 폭의 절반 - floor(W/2) 두께로) 값을 덧대어 채워주는 것
    - 이를 통해 합성곱 연산을 해도 결과 양상이 원양상과 크기가 같도록 할 수 있음

- 스트라이드(stride, 보폭)
  - 필터가 적용되는 영역을 **윈도우(window)**라고 부름
  - 스트라이드 : 다음 필터링을 위해 윈도우를 움직이는 거리.
  - s(stride) = 1 or 2 이상. 클 수록 영상의 크기 빨리 줄어듦

#9.6 오히려 괜찮은 모델을 만드는 풀링
> 풀링(pooling) : 이미지의 일정 영역 내 픽셀들이 가진 값을 하나로 축소하는 연산 (= 한 개의 대표값 선택)

- 풀링은 합성곱과 달리 **윈도우를 중첩시켜 이동하지 않음**
- 풀링의 결과값
  - 최대값 풀링(max pooling)
    - 풀링 적용 영역 내 가장 큰 값을 결과로 선택
  - 평균값 풀링(mean pooling)
    - 영역 평균값을 선택
- 풀링에 따른 영상 축소
  - 비례로 크기가 줄어들지만 패딩 없이 합성곱 연산한 것과 다르다 -> 주변부가 축소됨
    - 합성곱 연산에 의한 축소로 영상 주변부 정보를 잃게 되지만, 풀링은 다수의 픽셀 정보를 통합해 하나로 만드는 것.
    - 원래 신호에 존재하는 잡음 요소를 제거하는 효과
  - 풀링은 최대값 추출이나 평균값 추출과 같이 미리 정해진 기능을 수행하므로 학습 단계에서 모수를 최적화할 필요가 없는 계층.
    - 원래 이미지가 가지고 있던 채널 규대로 유지, 정보량(공간)만 압축
    - 다수의 픽셀 정보를 통합해 하나로 압축 -> 정보 요약, 이 과정에서 입력의 변화에 대해 덜 민감한 신호 전달
- 픽셀의 값을 일부 변경해도 최대값 풀링의 결과는 잘 바뀌지 않음
  - 이동 불변성이라 하며, 이동 불변성을 가진 모델을 강건한(robust) 모델이라고 함


> 인공 신경망에서 합성곱 연산을 수행하는 계층을 합성곱 계층, 풀링 연산을 수행하는 계층을 풀링 계층이라고 함.

-> 이를 번갈아 쌓아 ANN을 구성하면 신경망 모수의 수 효율적으로 줄임. 모델 복잡도 감소




케라스 API는 밀집 연결이나 평탄화 계층처럼 합성곱과 풀링을 위한 계층을 미리 만들어 제공한다.
- 다음과 같이 Sequential 클래스를 이용해 모델을 하나 만들고 합성곱과 풀링 계층을 추가


```
model = keras.models.Sequential( [
  keras.models.Conv2D(input_shape = (64, 64, 3),  kernel_size = (3, 3), filters = 32),
  keras.layers.MaxPooling2D((2, 2), strides=2),
  keras.layers.Conv2D(kernel_size = (3,3), padding='same', filters = 64),
  ...
])
```



# 9.9 전이학습 - 이미 훈련된 모델을 고쳐 쓰기
> 이미 학습이 완료된 모델을 다른 목적에 맞춰 조금만 고쳐서 사용하는 것.

- 이미 학습된 모델을 가져다가 특징 추출하는 용도로 사용
  - 그 결과를 이용해 우리가 원하는 문제 해결/작업 수행
  - 프로세스
    - 일부 계층을 잘라서 가져오고, 이 계층이 가지는 모수들은 변경되지 않도록
    - 후속 확장 계층은 새로운 작업에 맞는 출력을 내도록 구성하여 붙인다
      - 확장 부분이 학습으로 모수가 변경될 수 있다.
      - 사전 학습된 모델의 일부를 학습할 수도 있다.

In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf

# InceptionV3모델을 가져오면서 imagenet 데이터를 활용해 학습한 파라미터 가져옴
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(input_shape = (75, 75,  3),
                                include_top = False,
                                weights = 'imagenet') #반드시 훈련된 값을 사용

# 사전 학습된 모델의 모든 계층에 대해 파라미터를 고정시켜 둔다.
  # trainable 옵션을 False로 지정하면 됨.

for layer in pre_trained_model.layers:
  layer.trainable = False



87910968/87910968 [==============================] - 1s 0us/step
